In [1]:
from collections import deque

import numpy as np
import progressbar as pb

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import gym

from environments import *

from stable_baselines3.common.cmd_util import make_vec_env

c:\Users\ssuro\anaconda3\envs\torch\lib\site-packages\torch\utils\tensorboard\__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
c:\Users\ssuro\anaconda3\envs\torch\lib\site-packages\stable_baselines3\common\cmd_util.py:5: FutureWarning: Module ``common.cmd_util`` has been renamed to ``common.env_util`` and will be removed in the future.
  warnings.warn(


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
NUM_AGENTS = 16

In [4]:
env = make_vec_env('Reacher-v4', n_envs=NUM_AGENTS)

c:\Users\ssuro\anaconda3\envs\torch\lib\site-packages\gym\core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
c:\Users\ssuro\anaconda3\envs\torch\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [5]:
# reset the environment
env_info = env.reset()

# size of each action
action_size = env.action_space.shape[0]
print('Size of each action:', action_size)

state_size = env.observation_space.shape[0]
print('Observation size: {}'.format(state_size))

Size of each action: 2
Observation size: 11


In [6]:
def interact(action):
    action = action.reshape(NUM_AGENTS, action_size)
    next_state, reward, done, info = env.step(action)
    return next_state.reshape(NUM_AGENTS, -1), np.array(reward).reshape(NUM_AGENTS, -1), np.array(done).reshape(NUM_AGENTS, -1)

def reset():
    state = env.reset().reshape(NUM_AGENTS, -1)
    return state

In [7]:
NET_SIZE = 128

class Policy(nn.Module):
    
    def __init__(self, state_size, action_size=1, n_agents=1, fc1_size=NET_SIZE, fc2_size=NET_SIZE):
        super(Policy, self).__init__()
        
        self.bn0 = nn.BatchNorm1d(state_size)
        self.fc1 = nn.Linear(state_size, fc1_size)
        self.bn1 = nn.BatchNorm1d(fc1_size)
        self.fc2 = nn.Linear(fc1_size, fc2_size)
        self.bn2 = nn.BatchNorm1d(fc1_size)
        self.fc3_mu = nn.Linear(fc2_size, action_size)
        self.fc3_std = nn.Linear(fc2_size, action_size)

    def forward(self, state, log_std_min=-20, log_std_max=2):
        x = self.bn0(state)
        x = torch.relu(self.bn1(self.fc1(state)))
        x = torch.relu(self.bn2(self.fc2(x)))

        mean = self.fc3_mu(x)
        std = self.fc3_std(x)
        std = torch.clamp(std, log_std_min, log_std_max).exp()

        return mean, std
    
class Value(nn.Module):
    
    def __init__(self, state_size, action_size=1, n_agents=1, fc1_size=NET_SIZE, fc2_size=NET_SIZE):
        
        super(Value, self).__init__()
        
        self.bn0 = nn.BatchNorm1d(state_size)
        self.fc1 = nn.Linear(state_size, fc1_size)
        self.fc2 = nn.Linear(fc1_size, fc2_size)
        self.fc3 = nn.Linear(fc2_size, 1)
        
    def forward(self, x):
        x = self.bn0(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)
    
class Q(nn.Module):
    
    def __init__(self, state_size, action_size, n_agents=1, fc1_size=NET_SIZE, fc2_size=NET_SIZE):
        
        super(Q, self).__init__()
        
        self.bn0 = nn.BatchNorm1d(state_size+action_size)
        self.fc1 = nn.Linear(state_size + action_size, fc1_size)
        self.fc2 = nn.Linear(fc1_size, fc2_size)
        self.fc3 = nn.Linear(fc2_size, 1)
        
    def forward(self, s, a):
        x = torch.cat([s, a], 1)
        x = self.bn0(x)
        x = torch.relu(self.fc1(x)) 
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

In [8]:
from algorithms.sac import Agent

agent = Agent(
    state_size=state_size, 
    action_size=action_size,
    policy_network=Policy,
    value_network=Value,
    q_network=Q,
    n_agents=NUM_AGENTS, 
    device=device,
)

In [9]:
# agent.policy_network.load_state_dict(torch.load("./trained_models/g1-forward-homebrew/POLICY.pth"))
# agent.value_network_local.load_state_dict(torch.load("./trained_models/g1-forward-homebrew/VALUE_TARGET.pth"))
# agent.value_network_target.load_state_dict(torch.load("./trained_models/g1-forward-homebrew/VALUE_TARGET.pth"))
# agent.q_network_1.load_state_dict(torch.load("./trained_models/g1-forward-homebrew/Q_1.pth"))
# agent.q_network_2.load_state_dict(torch.load("./trained_models/g1-forward-homebrew/Q_2.pth"))

In [10]:
def run(n_episodes, t_max, print_every):
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=print_every)  # last 100 scores
    widget = ['training loop: ', pb.Percentage(), ' ', pb.Bar(), ' ', pb.ETA()]
    timer = pb.ProgressBar(widgets=widget, maxval=n_episodes).start()
    
    frame_counter = 0
    
    for i_episode in range(1, n_episodes+1):
        states = reset()
        score = 0
        for t in range(t_max):
            frame_counter += 1
            actions = agent.act(states)
            next_states, rewards, dones = interact(actions)
            agent.step(states, actions, rewards, next_states, dones)
            states = next_states
            score += rewards.mean()
            if np.any(dones):
                break 
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        agent.writer.add_scalar('score/mean', score, i_episode)
        if i_episode % print_every == 0:
            print('\rEpisode {}\tScore Mean: {:.2f}\tScore STD: {:.2f}'.format(i_episode, np.mean(scores_window), np.std(scores_window)))
            
        timer.update(i_episode)
    return scores

In [11]:
# torch.autograd.set_detect_anomaly(True)
%time scores = run(t_max=int(50), n_episodes=int(1000), print_every=100)

Episode 100	Score Mean: -3908174.93	Score STD: 749930.86


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

In [ ]:
torch.save(agent.policy_network.state_dict(), "./trained_models/g1-forward-homebrew/POLICY.pth")
torch.save(agent.value_network_target.state_dict(), "./trained_models/g1-forward-homebrew/VALUE_TARGET.pth")
torch.save(agent.q_network_1.state_dict(), "./trained_models/g1-forward-homebrew/Q_1.pth")
torch.save(agent.q_network_2.state_dict(), "./trained_models/g1-forward-homebrew/Q_2.pth")

In [ ]:
env.close()

In [ ]:
from gym.wrappers import RecordVideo

env = RecordVideo(gym.make('G1Dist-v0'), './video')
state = env.reset()
done = False
while not done:

    state = torch.from_numpy(state.reshape(NUM_AGENTS, -1)).float().to(device)

    agent.policy_network.eval()
    action, _ = agent.sample_action(state)
    state, reward, done, info = env.step(action.detach().cpu().numpy())
    
env.close()